In [1]:
!ls

bengin_traffic.csv  sample_data


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import pickle
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard

In [0]:
df_bening = pd.read_csv('bengin_traffic.csv')

In [4]:
df_bening.shape

(40395, 115)

In [0]:
#df_bening = (df_bening - df_bening.mean()) / df_bening.std()
#df_bening = scale(df_bening)

In [0]:
scaler = StandardScaler()
df_bening = scaler.fit_transform(df_bening)

In [0]:
x_train, x_opt, x_test = np.split(df_bening, [int((1/3)*len(df_bening)), int((2/3)*len(df_bening))])

In [37]:
print(x_train.shape)
print(x_opt.shape)
print(x_test.shape)

(13465, 115)
(13465, 115)
(13465, 115)


In [0]:
nb_epoch = 800
input_dim = x_train.shape[1]
learning_rate = 0.012

autoencoder = Sequential()
autoencoder.add(Dense(int(0.75 * input_dim), activation="tanh", input_dim=input_dim))
autoencoder.add(Dense(int(0.5 * input_dim), activation="tanh"))
autoencoder.add(Dense(int(0.33 * input_dim), activation="tanh"))
autoencoder.add(Dense(int(0.25 * input_dim), activation="tanh"))
autoencoder.add(Dense(int(0.33 * input_dim), activation="tanh"))
autoencoder.add(Dense(int(0.5 * input_dim), activation="tanh"))
autoencoder.add(Dense(int(0.75 * input_dim), activation="tanh"))
autoencoder.add(Dense(input_dim))

In [39]:
autoencoder.compile(loss='mean_squared_error',
                    optimizer='sgd')

cp = ModelCheckpoint(filepath="autoencoder_traffic.h5",
                               save_best_only=True,
                               verbose=0)

tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)

history = autoencoder.fit(x_train, x_train,
                    epochs=nb_epoch,
                    validation_data=(x_opt, x_opt),
                    verbose=1,
                    callbacks=[cp, tb]).history

Train on 13465 samples, validate on 13465 samples
Epoch 1/800
13465/13465 [==============================] - 1s 109us/step - loss: 1.0663 - val_loss: 0.7341
Epoch 2/800
13465/13465 [==============================] - 1s 90us/step - loss: 0.9301 - val_loss: 0.6533
Epoch 3/800
13465/13465 [==============================] - 1s 90us/step - loss: 0.8739 - val_loss: 0.6169
Epoch 4/800
13465/13465 [==============================] - 1s 90us/step - loss: 0.8442 - val_loss: 0.5960
Epoch 5/800
13465/13465 [==============================] - 1s 90us/step - loss: 0.8254 - val_loss: 0.5825
Epoch 6/800
13465/13465 [==============================] - 1s 89us/step - loss: 0.8120 - val_loss: 0.5724
Epoch 7/800
13465/13465 [==============================] - 1s 89us/step - loss: 0.8010 - val_loss: 0.5639
Epoch 8/800
13465/13465 [==============================] - 1s 89us/step - loss: 0.7911 - val_loss: 0.5560
Epoch 9/800
13465/13465 [==============================] - 1s 91us/step - loss: 0.7818 - val_loss: 0.

In [40]:
plt.plot(history['loss'], linewidth=2, label='Train')
plt.plot(history['val_loss'], linewidth=2, label='Test')
plt.legend(loc='upper right')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [0]:
x_opt_predictions = autoencoder.predict(x_opt)

In [0]:
mse = np.mean(np.power(x_opt - x_opt_predictions, 2), axis=1)

In [53]:
mse.shape

(13465,)

In [55]:
tr = mse.mean() + mse.std()
tr

7.455034536099385